#**Crawling Data Komentar Youtube**



#1. Import Library

In [1]:
import pandas as pd
from googleapiclient.discovery import build

#2. Fungsi untuk crawling komentar

In [2]:
def video_comments(video_id):
	# empty list for storing reply
	replies = []

	# creating youtube resource object
	youtube = build('youtube', 'v3', developerKey=api_key)

	# retrieve youtube video results
	video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()

	# iterate video response
	while video_response:

		# extracting required info
		# from each result object
		for item in video_response['items']:

			# Extracting comments ()
			published = item['snippet']['topLevelComment']['snippet']['publishedAt']
			user = item['snippet']['topLevelComment']['snippet']['authorDisplayName']

			# Extracting comments
			comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
			likeCount = item['snippet']['topLevelComment']['snippet']['likeCount']

			replies.append([published, user, comment, likeCount])

			# counting number of reply of comment
			replycount = item['snippet']['totalReplyCount']

			# if reply is there
			if replycount>0:
				# iterate through all reply
				for reply in item['replies']['comments']:

					# Extract reply
					published = reply['snippet']['publishedAt']
					user = reply['snippet']['authorDisplayName']
					repl = reply['snippet']['textDisplay']
					likeCount = reply['snippet']['likeCount']

					# Store reply is list
					#replies.append(reply)
					replies.append([published, user, repl, likeCount])

			# print comment with list of reply
			#print(comment, replies, end = '\n\n')

			# empty reply list
			#replies = []

		# Again repeat
		if 'nextPageToken' in video_response:
			video_response = youtube.commentThreads().list(
					part = 'snippet,replies',
					pageToken = video_response['nextPageToken'],
					videoId = video_id
				).execute()
		else:
			break
	#endwhile
	return replies


#3. Jalankan Proses Crawling

In [3]:
# isikan dengan api key Anda
api_key = 'API KEY'

# Enter video id
# contoh url video = https://youtu.be/dMj1-G2fM7E
video_id = "dMj1-G2fM7E" #isikan dengan kode / ID video

# Call function
comments = video_comments(video_id)

comments

[['2024-10-13T02:19:37Z',
  '@baradragneel3681',
  'OPPO Indonesia : Open Recruitment for CMO Oppo Indonesia',
  0],
 ['2024-10-13T02:10:42Z', '@xopurwanto-d9q', '🤣', 0],
 ['2024-10-13T02:06:26Z', '@baqusputra', 'Kelihatan kualitasnya wkwk', 0],
 ['2024-10-13T02:03:14Z',
  '@nanako_12',
  '<a href="https://www.youtube.com/watch?v=dMj1-G2fM7E&amp;t=26">0:26</a> <a href="https://www.youtube.com/watch?v=dMj1-G2fM7E&amp;t=31">0:31</a> 😂😂',
  0],
 ['2024-10-13T02:02:29Z', '@FawBio', 'Sampah', 0],
 ['2024-10-13T02:01:21Z',
  '@leszaleonardo6688',
  'Oppo clearly needs to sign another Indonesian CMO ... the product itself is not bad, but he clearly needs to resign ... what a joke ...',
  0],
 ['2024-10-13T01:46:39Z',
  '@vaingloryhell321',
  'Wohahahahaha brand hp paling bnyak rusak di konter perbaikan smartphone.😂😂😂  Lawak.',
  1],
 ['2024-10-13T01:33:35Z',
  '@refanto_doank',
  'ga pernah tertarik ama smartphone dari brand ini',
  1],
 ['2024-10-13T01:31:45Z',
  '@rikiamirkusnadi859',
  'Th

#4. Ubah Hasil Crawling ke Dataframe

In [4]:
df = pd.DataFrame(comments, columns=['publishedAt', 'authorDisplayName', 'textDisplay', 'likeCount'])
df

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2024-10-13T02:19:37Z,@baradragneel3681,OPPO Indonesia : Open Recruitment for CMO Oppo...,0
1,2024-10-13T02:10:42Z,@xopurwanto-d9q,🤣,0
2,2024-10-13T02:06:26Z,@baqusputra,Kelihatan kualitasnya wkwk,0
3,2024-10-13T02:03:14Z,@nanako_12,"<a href=""https://www.youtube.com/watch?v=dMj1-...",0
4,2024-10-13T02:02:29Z,@FawBio,Sampah,0
...,...,...,...,...
2176,2024-10-09T10:51:53Z,@Qhof17,"​@@teguhsentosa49 Liat aja <a href=""https://ww...",4
2177,2024-10-09T10:53:35Z,@ridhooktaviansyaputra2771,@@teguhsentosa49macbook jelas 😂😂,0
2178,2024-10-08T07:42:14Z,@muhamadwahyupamuji2063,"Mampir dulu ahhhhh, siapa tau rame nantiiiii",1
2179,2024-10-08T06:43:36Z,@priyonugroho7865,bandinginnya jangan sama processor 2 tahun yan...,2


#5. Simpan Hasil Crawling ke file CSV

In [5]:
df.to_csv('youtube-comments.csv', index=False)